In [244]:
#Simplified Sudoku Simulated Annealing with Polish#
import sys, math, random
from copy import deepcopy

sys.path.append(r'C:\Users\gavin\OneDrive\Desktop\Artificial Intelligence')
from sudoku_parser import SudokuPuzzle

def build_fixed(puzzle):
    return [[cell != 0 for cell in row] for row in puzzle.grid]

def cells_in_block(br, bc):
    return ((r, c) for r in range(br*3, br*3+3) for c in range(bc*3, bc*3+3))

def populate_random_block(puzzle, fixed):
    for br in range(3):
        for bc in range(3):
            have = {puzzle.grid[r][c] for r, c in cells_in_block(br, bc) if fixed[r][c]}
            empties = [(r, c) for r, c in cells_in_block(br, bc) if not fixed[r][c]]
            for (r, c), val in zip(empties, random.sample([d for d in range(1,10) if d not in have], len(empties))):
                puzzle.grid[r][c] = val

def conflicts(puzzle):
    return sum(9 - len(set([puzzle.grid[r][c] for c in range(9)])) for r in range(9)) + \
           sum(9 - len(set([puzzle.grid[r][c] for r in range(9)])) for c in range(9))

def block_valid(puzzle, br, bc):
    vals = [puzzle.grid[r][c] for r, c in cells_in_block(br, bc)]
    return set(vals) == set(range(1,10))

def is_solved(puzzle):
    return all(len(set([puzzle.grid[r][c] for c in range(9)])) == 9 for r in range(9)) and \
           all(len(set([puzzle.grid[r][c] for r in range(9)])) == 9 for c in range(9)) and \
           all(block_valid(puzzle, br, bc) for br in range(3) for bc in range(3))

def get_conflicts(puzzle):
    bad = set()
    for r in range(9):
        counts = {v: sum(1 for c in range(9) if puzzle.grid[r][c]==v) for v in range(1,10)}
        for c in range(9):
            if counts[puzzle.grid[r][c]] > 1:
                bad.add((r,c))
    for c in range(9):
        counts = {v: sum(1 for r in range(9) if puzzle.grid[r][c]==v) for v in range(1,10)}
        for r in range(9):
            if counts[puzzle.grid[r][c]] > 1:
                bad.add((r,c))
    return bad

def best_block_swap(puzzle, fixed, r, c, k=6):
    if fixed[r][c]: return None
    br, bc = r//3, c//3
    candidates = [(rr,cc) for rr,cc in cells_in_block(br,bc) if not fixed[rr][cc] and (rr,cc)!=(r,c)]
    if not candidates: return None
    sample = random.sample(candidates, min(k,len(candidates)))
    base = conflicts(puzzle)
    best = None
    for rr,cc in sample:
        puzzle.grid[r][c], puzzle.grid[rr][cc] = puzzle.grid[rr][cc], puzzle.grid[r][c]
        new = conflicts(puzzle)
        puzzle.grid[r][c], puzzle.grid[rr][cc] = puzzle.grid[rr][cc], puzzle.grid[r][c]
        if best is None or new < best[4]:
            best = (r,c,rr,cc,new,base)
    return best

def apply_swap(puzzle, move): r1,c1,r2,c2,_,_ = move; puzzle.grid[r1][c1], puzzle.grid[r2][c2] = puzzle.grid[r2][c2], puzzle.grid[r1][c1]

def polish(puzzle, fixed, passes=5):
    for _ in range(passes):
        changed=False
        for br in range(3):
            for bc in range(3):
                vals=[puzzle.grid[r][c] for r,c in cells_in_block(br,bc)]
                positions=[(r,c) for r,c in cells_in_block(br,bc)]
                counts={v:vals.count(v) for v in vals}
                duplicates=[v for v,cnt in counts.items() if cnt>1]
                missing=[v for v in range(1,10) if v not in counts]
                if not duplicates: continue
                for (r,c),v in zip(positions,vals):
                    if fixed[r][c]: continue
                    if v in duplicates and missing:
                        puzzle.grid[r][c] = missing.pop(0)
                        duplicates.remove(v)
                        changed=True
        if not changed: break

def anneal(puzzle, fixed, max_steps=250_000, T0=4.0, cooling=0.9992, log_every=5000):
    populate_random_block(puzzle, fixed)
    best_grid = deepcopy(puzzle.grid)
    best_cost = current = conflicts(puzzle)
    T=T0
    for step in range(max_steps):
        if current==0:
            polish(puzzle,fixed)
            if is_solved(puzzle): print(f"✅ Solved in {step} steps!"); return True
        conflicts_cells = list(get_conflicts(puzzle))
        if conflicts_cells:
            r,c = random.choice(conflicts_cells)
        else:
            br,bc = random.randint(0,2), random.randint(0,2)
            r,c = random.choice(list(cells_in_block(br,bc)))
        move = best_block_swap(puzzle,fixed,r,c)
        if move:
            _,_,_,_,new,base = move
            delta = new-base
            if delta<=0 or random.random()<math.exp(-delta/max(T,1e-9)):
                apply_swap(puzzle,move)
                current=new
                if new<best_cost:
                    best_cost=new
                    best_grid=deepcopy(puzzle.grid)
        T*=cooling
        if log_every and step%log_every==0:
            print(f"Step {step} | Conflicts={current} | Best={best_cost}")
    puzzle.grid = best_grid
    polish(puzzle,fixed)
    solved=is_solved(puzzle)
    print("Solved!" if solved else "Max steps reached.")
    return solved

def solve(file_path, attempts=5, seed=None):
    if seed: random.seed(seed)
    for i in range(attempts):
        print(f"\n🔁 Attempt {i+1}")
        puzzle=SudokuPuzzle(file_path)
        fixed=build_fixed(puzzle)
        puzzle.display()
        if anneal(puzzle,fixed):
            puzzle.display()
            print("Solved!")
            return
    print("All attempts failed.")

if __name__=="__main__":
    solve(r'C:\Users\gavin\OneDrive\Desktop\Artificial Intelligence\Med-P2.txt')



🔁 Attempt 1
? 7 9 4 3 ? 1 ? ?
3 ? ? ? ? ? 6 ? ?
8 5 ? ? 6 7 ? 3 ?
? ? ? ? ? 6 ? 8 ?
? ? 5 ? 4 ? 7 ? ?
? 8 ? 5 ? ? ? ? ?
? 9 ? 1 7 ? ? 6 3
? ? 8 ? ? ? ? ? 1
? ? 6 ? 9 4 2 7 ?
Step 0 | Conflicts=41 | Best=41
✅ Solved in 1010 steps!
6 7 9 4 3 5 1 2 8
3 4 2 9 8 1 6 5 7
8 5 1 2 6 7 9 3 4
4 1 3 7 2 6 5 8 9
9 6 5 3 4 8 7 1 2
2 8 7 5 1 9 3 4 6
5 9 4 1 7 2 8 6 3
7 2 8 6 5 3 4 9 1
1 3 6 8 9 4 2 7 5
Solved!
